
# 0055 回測模型
## 2007~2018
> 策略1：每年投入固定資金，不賣，下跌x%買i股
> 策略2：除權息後買不賣

In [352]:
# 讀取csv(資料來源XQ匯入excel調整)
#['時間(yyyy/mm/dd)', '開盤價', '最高價', '最低價', '收盤價', '漲跌幅(%)', '成交量']
import csv
temp=[]
temp_all=[[] for i in range(19)]
with open('0055.csv') as cf:
    rows = csv.reader(cf)
    for row in rows:
        temp.append(row)
        
# str->float
for i in range(len(temp)):
    for j in range(len(temp[i])):
        hh = temp[i][j]
        if type(hh)==str:
            try:
                temp[i][j]=float(hh)
            except:
                pass
            
## 以年分割資料(2007年以後)
itertemp = iter(temp)
next(itertemp)
for i in itertemp:
    year = int(i[0][0:4])-2007
    temp_all[year].append(i)
    
#除權息和金額    
dividend={
"2017/11/23":0.5,
"2016/11/23":0.4,
"2015/11/24":0.4,
"2014/11/25":0.4,
"2013/11/25":0.3,
"2012/11/22":0.2,
"2011/11/23":0.3,
"2010/11/23":0.4
}

In [353]:
# 測試股利
import doctest
def test_dividend():
    """
    >>> '2010/11/23' in dividend.keys()
    True
    >>> dividend["2010/11/23"]
    0.4
    """
doctest.testmod()


TestResults(failed=0, attempted=2)

In [354]:
def findbest(start_money,lower,lower_buy,temp):
    
    ##初始化金額、股數、flag
    money=start_money
    stock = 0
    flag=False
    
    # 如果傳的是年就合併所有資料
    if len(temp) < 30:
        flag=True
        test=[]
        for i in temp:
            test.extend(i)
        temp=test
    
    ## 從有漲跌幅的開始到結束
    count=1
    for row in temp[2:-1]:
        
        #每年投入start_money
        count+=1
        if count%250==0:
            money+=start_money
            
        #除權息
        if row[0] in dividend.keys():
#             print("===\n new money\t{}+{}股*{}元\n===".format(money,stock,dividend[row[0]]))
            money+=stock*dividend[row[0]]
           
            
                   
        #跌買,花錢
        if row[5]<-1*lower and money>lower_buy*row[4]:
            stock+=lower_buy
            money-=row[4]*lower_buy
            if flag:
                print("{}買進\t剩餘:{:.0f}元\t剩餘:{}股".format(row[0],money,stock))
        
    total = int(money+row[4]*stock)
    
    #年回測
    if flag:
        print("-------------------------------------")
        print("如果漲跌{}%,買{}股".format(lower,lower_buy))
        print("結:total={},money={},stock={}".format(total,money,stock))
        print("淨利:%d萬\t總投入%d萬"%((total-int(count/250)*start_money)/10000,(int(count/250)*start_money)/10000))
        

    #全部回測，而且只取賺的,return 總投入(萬),總金額(萬),現金(萬),股數  
    elif total > 0:
        return [int(count/250)*start_money/10000,total/10000,money/10000,stock]

In [260]:
print("Find best parameter")
print("===================")
##跑回側
# x=每期投入幾萬元(range)
# i*10=lowerbound
# j*100=buy
data=[]
for x in range(10,31,1):
    money=x*10000
    earn_max=[0,0,0,0]
    best_parameter=[0,0]
    for i in range(0,31,1):
        for j in range(0,31,1):
            try:
                earn=findbest_2(money,i/10,j*100,temp)
                if earn_max[1]<earn[1]:
                    earn_max=earn.copy()
                    best_parameter=[i/10,j*100]
            except:
                pass
                        
    data.append([x,best_parameter,earn_max])
    
    print("如果跌{0[0]}%,買{0[1]}股".format(best_parameter))
    print("結:total={0[1]}萬元,money={0[2]}萬元,stock={0[3]}".format(earn_max))
    print("起始資金{}萬元，總投入{}萬元".format(x,int(earn_max[0])))
    print('---------------------------------------------')

Find best parameter
如果跌0.0%,買100股
結:total=177.3949萬元,money=0.005999999999999978萬元,stock=99100
起始資金10萬元，總投入100萬元
---------------------------------------------
如果跌0.0%,買100股
結:total=194.2833萬元,money=0.2474萬元,stock=108400
起始資金11萬元，總投入110萬元
---------------------------------------------
如果跌0.0%,買200股
結:total=211.971萬元,money=0.035萬元,stock=118400
起始資金12萬元，總投入120萬元
---------------------------------------------
如果跌0.0%,買200股
結:total=229.9912萬元,money=0.1552萬元,stock=128400
起始資金13萬元，總投入130萬元
---------------------------------------------
如果跌0.0%,買200股
結:total=248.1658萬元,money=0.0718萬元,stock=138600
起始資金14萬元，總投入140萬元
---------------------------------------------
如果跌0.0%,買200股
結:total=266.5782萬元,money=0.22620000000000004萬元,stock=148800
起始資金15萬元，總投入150萬元
---------------------------------------------
如果跌0.0%,買200股
結:total=284.8192萬元,money=0.2092萬元,stock=159000
起始資金16萬元，總投入160萬元
---------------------------------------------
如果跌0.0%,買200股
結:total=303.3063萬元,money=0.0804萬元,stock=169400
起始資金17萬元，總投入170萬元
--

In [355]:
## 統計
from collections import Counter
test=[str(i[1]) for i in data]
counter=Counter(test)
counter

Counter({'[0.0, 100]': 2, '[0.0, 200]': 9, '[0.0, 300]': 9, '[0.0, 400]': 1})

In [356]:
#年回測
findbest(120000,0,200,temp_all[0:-1])

2007/7/19買進	剩餘:116740元	剩餘:200股
2007/7/25買進	剩餘:113460元	剩餘:400股
2007/7/26買進	剩餘:110238元	剩餘:600股
2007/7/27買進	剩餘:107168元	剩餘:800股
2007/7/30買進	剩餘:104158元	剩餘:1000股
2007/8/1買進	剩餘:101198元	剩餘:1200股
2007/8/2買進	剩餘:98268元	剩餘:1400股
2007/8/6買進	剩餘:95388元	剩餘:1600股
2007/8/7買進	剩餘:92512元	剩餘:1800股
2007/8/10買進	剩餘:89664元	剩餘:2000股
2007/8/13買進	剩餘:86824元	剩餘:2200股
2007/8/15買進	剩餘:84076元	剩餘:2400股
2007/8/16買進	剩餘:81400元	剩餘:2600股
2007/8/21買進	剩餘:78652元	剩餘:2800股
2007/8/27買進	剩餘:75838元	剩餘:3000股
2007/8/28買進	剩餘:73066元	剩餘:3200股
2007/8/29買進	剩餘:70334元	剩餘:3400股
2007/9/4買進	剩餘:67538元	剩餘:3600股
2007/9/5買進	剩餘:64778元	剩餘:3800股
2007/9/10買進	剩餘:62034元	剩餘:4000股
2007/9/13買進	剩餘:59262元	剩餘:4200股
2007/9/17買進	剩餘:56466元	剩餘:4400股
2007/9/20買進	剩餘:53634元	剩餘:4600股
2007/9/21買進	剩餘:50834元	剩餘:4800股
2007/10/4買進	剩餘:47806元	剩餘:5000股
2007/10/5買進	剩餘:44790元	剩餘:5200股
2007/10/9買進	剩餘:41754元	剩餘:5400股
2007/10/12買進	剩餘:38798元	剩餘:5600股
2007/10/17買進	剩餘:35834元	剩餘:5800股
2007/10/19買進	剩餘:32880元	剩餘:6000股
2007/10/22買進	剩餘:29992元	剩餘:6200股
2007/10/24買進	剩餘:27102元	剩餘:6400股
2007/10

2018/3/14買進	剩餘:4670元	剩餘:118200股
2018/3/15買進	剩餘:1294元	剩餘:118400股
-------------------------------------
如果漲跌0%,買200股
結:total=2120654,money=1294.0,stock=118400
淨利:92萬	總投入120萬


# 除權息後買

In [357]:
buy=[
[12.14,0.40],
[10.12,0.30],
[10.41,0.20],
[13.75,0.30],
[14.9,0.40],
[13.83,0.40],
[14.02,0.40],
[16.32,0.50]
]

In [358]:
# 2010/7/6
class Test():
 
    def __init__(self,add_money):
        self.add_money = add_money
        self.stock = 0
        #2007~2010
        self.money = add_money*4
        self.buy=[
        [12.14,0.40],
        [10.12,0.30],
        [10.41,0.20],
        [13.75,0.30],
        [14.9,0.40],
        [13.83,0.40],
        [14.02,0.40],
        [16.32,0.50]]
    
    def go(self):
        
        for i in range(len(self.buy)):
            self.money += self.stock*self.buy[i][1]
            buystock = int(self.money/self.buy[i][0]/1000)*1000
            self.stock += buystock
            self.money = self.money-buystock*self.buy[i][0]+self.add_money
            
            
    def total(self):
        
        #2018
        self.money+=add_money
        print("結:total={}元,money={}元,stock={}".format(self.stock*18+self.money,self.money,self.stock))
        


In [359]:
test=Test(120000)
test.go()
test.total()

結:total=2368780.0元,money=244780.0元,stock=118000


# 除權息後買比較好